In [ ]:
import gradio as gr
from transformers import AutoImageProcessor
from transformers import SiglipForImageClassification
from transformers.image_utils import load_image
from PIL import Image
import torch

# Load model and processor
model_name = "prithivMLmods/Guard-Against-Unsafe-Content-Siglip2"
model = SiglipForImageClassification.from_pretrained(model_name)
processor = AutoImageProcessor.from_pretrained(model_name)

def nsfw_detection(image):
    """Predicts NSFW probability scores for an image."""
    image = Image.fromarray(image).convert("RGB")
    inputs = processor(images=image, return_tensors="pt")
    
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probs = torch.nn.functional.softmax(logits, dim=1).squeeze().tolist()
    
    labels = model.config.id2label
    predictions = {labels[i]: round(probs[i], 3) for i in range(len(probs))}
    
    return predictions

# Create Gradio interface
iface = gr.Interface(
    fn=nsfw_detection,
    inputs=gr.Image(type="numpy"),
    outputs=gr.Label(label="NSFW Content Detection"),
    title="NSFW Image Detection",
    description="Upload an image to check if it contains unsafe content such as vulgarity or nudity."
)

# Launch the app
if __name__ == "__main__":
    iface.launch()
